In [ ]:
!pip install openai
!pip install python-dotenv
!pip install google-generativeai
!pip install gtts

!pip install SpeechRecognition
!pip install pydub

In [4]:
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from gtts import gTTS
import os
import speech_recognition as sr

# 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI()

# 시스템 메시지
system = """
당신은 '롯데리아 수원영통점' 직원이고, 당신의 업무는 '매장 소개', '주문 받기', '결제 방식 안내', '인사' 총 4가지가 있어
먼저, '매장 소개'는 [매장 정보]를 토대로 손님들이 하는 질문들에 대답해주는 것
'주문 받기'는 손님이 [메뉴 정보]의 <햄버거>, <사이드>, <음료> 카테고리별로 음식을 담으면 카테고리별로 총 합 가격을 기억해뒀다가 마지막에 손님이 장바구니에 담긴 총합 가격을 물어보면 알려주는 것
'결제 방식 안내'는 [결제 방식]을 토대로 총 가격의 합을 어떠한 방식으로 결제할건지 질문하는 것
'인사'는 손님의 주문과 결제가 끝나면 정중하게 인사하는 것
----------------------------------
[매장 정보]
매장 이름 : 롯데리아 수원영통점
주소 : 경기 수원시 영통구 청명남로 21
영업 시간 : 매일 10:00 ~ 22:00
전화번호 : 031-206-0066
----------------------------------
[메뉴 정보]
<햄버거>
오징어 얼라이브 버거 (5,500원)
오징어 얼라이브 비프 버거 (5,700원)
전주비빔 라이스 버거 (6,900원)
...
[결제 방식]
신용/체크 카드 결제
상품권(바코드) 결제
현금 결제
"""

# 대화 기록을 저장할 전역 변수
messages = []

def process(prompt, history):
    global messages
    messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # 시스템 메시지를 추가
    messages.append({"role": "system", "content": system})

    response_text = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": response_text})    
    
    # gTTS를 사용하여 음성 생성
    tts = gTTS(response_text, lang='ko')
    tts.save("response.mp3")
    
    return "BOT: " + response_text, "response.mp3"  # 텍스트와 오디오 파일 경로 반환

def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio, language="ko-KR")
    except sr.UnknownValueError:
        return "음성을 인식할 수 없습니다."
    except sr.RequestError as e:
        return f"음성 인식 서비스에 접근할 수 없습니다; {e}"

with gr.Blocks() as demo:
    audio_input = gr.Audio(type="filepath", label="AI에게 음성 질문하기")
    audio_output = gr.Audio(label="AI의 음성 답변")
    text_input = gr.Textbox(label="텍스트로 질문하기")
    chatbot = gr.Chatbot(label="AI와의 대화")

    def respond(audio, text, history):
        if audio:
            text = transcribe_audio(audio)
        bot_response, audio_file = process(text, history)
        history.append((text, bot_response))
        return history, gr.update(value=audio_file, autoplay=True), ""

    audio_input.change(respond, [audio_input, text_input, chatbot], [chatbot, audio_output, text_input])
    text_input.submit(respond, [audio_input, text_input, chatbot], [chatbot, audio_output, text_input])

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a70f87af3ce0166cbf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
